# [DFA Preliminaries](./DFASlide.pdf)

In [1]:
nprocs()==1 && addprocs()

@everywhere logis(x) = 1 ./ (1 .+ exp(-x))
@everywhere dlogis(x) = logis(x) .* (1 .- logis(x))

@everywhere function initParams(x,y,n_hidden,max_iter)
    w1 = randn(n_hidden,size(x,1))
    w2 = randn(n_hidden,n_hidden)
    w3 = randn(size(y,1),n_hidden)  
    B1 = rand(n_hidden,1)
    B2 = rand(n_hidden,1)      
    errors = zeros(max_iter)
    outputs = zeros(size(y,2),max_iter)
    return (w1,w2,w3,errors,outputs,B1,B2)
end

@everywhere function ffward(x,y,w1,w2,w3)
    a1 = w1 * x
    h1 = logis(a1)
    a2 = w2 * h1
    h2 = logis(a2)
    ay = w3 * h2
    yhat = logis(ay)
    err = yhat - y
    return(a1,h1,a2,h2,ay,yhat,err)
end

@everywhere function updateW!(w1,w2,w3,x,h1,h2,δa1,δa2,err;lr=1)
    w1 .-= δa1 * x' .* lr
    w2 .-= δa2 * h1' .* lr
    w3 .-= err * h2' .* lr
end

@everywhere function bp(x,y,n_hidden,max_iter)
    (w1,w2,w3,errors,outputs,_,_)=initParams(x,y,n_hidden,max_iter)
    for i = 1:max_iter   
        (a1,h1,a2,h2,ay,yhat,err) = ffward(x,y,w1,w2,w3)
        δa2 = (w3' * err) .* dlogis(a2)
        δa1 = (w2' * δa2) .* dlogis(a1)
        updateW!(w1,w2,w3,x,h1,h2,δa1,δa2,err)
        errors[i] = sum(abs(err))
        outputs[:,i] = yhat
    end
    return (outputs,errors)
end

@everywhere function fa(x,y,n_hidden,max_iter)
    (w1,w2,w3,errors,outputs,B1,B2)=initParams(x,y,n_hidden,max_iter)
    for i = 1:max_iter    
        (a1,h1,a2,h2,ay,yhat,err) = ffward(x,y,w1,w2,w3)
        δa2 = (B2.*err) .* dlogis(a2) 
        δa1 = (B1.*δa2) .* dlogis(a1) 
        updateW!(w1,w2,w3,x,h1,h2,δa1,δa2,err)
        errors[i] = sum(abs(err))
        outputs[:,i] = yhat
    end
    return (outputs,errors)
end

@everywhere function dfa(x,y,n_hidden,max_iter)
    (w1,w2,w3,errors,outputs,B1,B2)=initParams(x,y,n_hidden,max_iter)
    for i = 1:max_iter    
        (a1,h1,a2,h2,ay,yhat,err) = ffward(x,y,w1,w2,w3)
        δa2 = (B2.*err) .* dlogis(a2) 
        δa1 = (B1.*err) .* dlogis(a1) 
        updateW!(w1,w2,w3,x,h1,h2,δa1,δa2,err)
        errors[i] = sum(abs(err))
        outputs[:,i] = yhat
    end
    return (outputs,errors)
end

@everywhere function ifa(x,y,n_hidden,max_iter)
    (w1,w2,w3,errors,outputs,B1,_)=initParams(x,y,n_hidden,max_iter)
    for i = 1:max_iter    
        (a1,h1,a2,h2,ay,yhat,err) = ffward(x,y,w1,w2,w3)
        δa1 = (B1.*err) .* dlogis(a1) 
        δa2 = (w2 * δa1) .* dlogis(a2) 
        updateW!(w1,w2,w3,x,h1,h2,δa1,δa2,err)
        errors[i] = sum(abs(err))
        outputs[:,i] = yhat
    end
    return (outputs,errors)
end



In [2]:
x = [1 1 ; 0 1 ; 1 0 ; 0 0]'
y = [0 1 1 0]
n_hidden = 15; # Number of hidden units
max_iter = 500; #Number of learning steps
trials = 30;

dfa_err=@parallel (hcat) for _=1:trials
    (_,dfa_errors) = dfa(x,y,n_hidden,max_iter)
    dfa_errors
end
(dfa_yhat,_) = dfa(x,y,n_hidden,max_iter)

bp_err = @parallel (hcat) for _=1:trials
    (_,bp_errors) = bp(x,y,n_hidden,max_iter)
    bp_errors
end
(bp_yhat,_) = bp(x,y,n_hidden,max_iter)

ifa_err=@parallel (hcat) for _=1:trials
    (_,ifa_errors) = ifa(x,y,n_hidden,max_iter)
    ifa_errors
end
(ifa_yhat,_) = ifa(x,y,n_hidden,max_iter);

fa_err=@parallel (hcat) for _=1:trials
    (_,fa_errors) = fa(x,y,n_hidden,max_iter)
    fa_errors
end
(fa_yhat,_) = fa(x,y,n_hidden,max_iter);

using Plots
l = @layout [
    a b
    c d
    e f
]
gr()
p1=plot(dfa_yhat',label=["1,1" "0,1" "1,0" "0,0"],title="DFA")
p2=plot(ifa_yhat',label=["1,1" "0,1" "1,0" "0,0"],title="IFA")
p3=plot(fa_yhat',label=["1,1" "0,1" "1,0" "0,0"],title="FA")
p4=plot(bp_yhat',label=["1,1" "0,1" "1,0" "0,0"],title="BP")
p5=plot([dfa_err[:,1] ifa_err[:,1] fa_err[:,1] bp_err[:,1]],label=["DFA" "IFA" "FA" "BP"],title="First Trial Error")
p6=plot([mean(dfa_err,2) mean(ifa_err,2) mean(fa_err,2) mean(bp_err,2)],label=["DFA" "IFA" "FA" "BP"],title="Mean Trials Error")
plot(p1,p2,p3,p4,p5,p6,size=(900,900),layout=l)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 500 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 DFA 
 
<polyline clip-path="url(#clip02)" style="stroke:#0099ff; stroke-width:1.8; stroke-opacity:1; fill:none" points="
 39.4926,214.568 40.3073,28.3872 41.1221,282.182 41.9369,47.7026 42.7516,280.403 43.5664,27.642 44.3812,283.39 45.196,26.2331 46.0107,283.276 46.8255,24.6174 
 47.6403,283.373 48.455,24.2636 49.2698,283.55 50.0846,24.1897 50.8993,283.688 51.7141,24.1716 52.5289,283.781 53.3437,24.1635 54.1584,283.839 54.9732,24.1535 
 55.788,283.872 56.6027,24.1385 57.4175,283.889 58.2323,24.1186 59.0471,283.896 59.8618,24.0944 60.6766,283.898 61.4914,24.0666 62.3061,283.899 63.1209,24.0356 
 63.9357,283.901 64.7504,24.0015 65.5652,283.905 66.38,23.9642 67.1948,283.911 68.0095,23.9235 68.8243,283.914 69.6391,23.8796 70.4538,283.911 71.2686,23.8329 
 72.0834,283.896 72.8981,23.7844 73.7129,283.863 74.5277,23.7354 75.3425,283.806 76.1572,23.6873 76.972,283.719 77.7868,23.6416 78.6015,283.592 79.4163,23.5995 
 80.2311,283.417 81.0459,23.5617 81.8606,283.181 82.6754,23.5289 83.4902,282.87 84.3049,23.5012 85.1197,282.467 85.9345,23.4785 86.7492,281.958 87.564,23.4605 
 88.3788,281.337 89.1936,23.4467 90.0083,280.612 90.8231,23.4367 91.6379,279.821 92.4526,23.43 93.2674,279.032 94.0822,23.4263 94.897,278.339 95.7117,23.4252 
 96.5265,277.832 97.3413,23.4268 98.156,277.568 98.9708,23.4312 99.7856,277.546 100.6,23.4389 101.415,277.714 102.23,23.451 103.045,277.981 103.859,23.4693 
 104.674,278.214 105.489,23.497 106.304,278.217 107.119,23.5421 107.933,277.614 108.748,23.6285 109.563,275.438 110.378,23.8584 111.192,268.644 112.007,24.753 
 112.822,254.507 113.637,27.1911 114.451,248.578 115.266,29.2292 116.081,249.35 116.896,30.7194 117.711,252.502 118.525,31.3079 119.34,256.032 120.155,31.4748 
 120.97,259.597 121.784,31.5143 122.599,263.007 123.414,31.5871 124.229,266.142 125.043,31.791 125.858,268.948 126.673,32.19 127.488,271.418 128.303,32.8369 
 129.117,273.568 129.932,33.7904 130.747,275.417 131.562,35.1307 132.376,276.987 133.191,36.9745 134.006,278.295 134.821,39.4954 135.635,279.362 136.45,42.9565 
 137.265,280.207 138.08,47.7722 138.895,280.841 139.709,54.6307 140.524,281.265 141.339,64.7693 142.154,281.44 142.968,80.6452 143.783,281.232 144.598,107.602 
 145.413,280.137 146.227,155.888 147.042,276.108 147.857,220.886 148.672,265.684 149.487,251.919 150.301,259.432 151.116,257.789 151.931,259.165 152.746,259.455 
 153.56,260.035 154.375,260.484 155.19,260.93 156.005,261.34 156.82,261.727 157.634,262.09 158.449,262.432 159.264,262.753 160.079,263.056 160.893,263.342 
 161.708,263.611 162.523,263.864 163.338,264.104 164.152,264.329 164.967,264.541 165.782,264.741 166.597,264.929 167.412,265.106 168.226,265.271 169.041,265.425 
 169.856,265.568 170.671,265.701 171.485,265.823 172.3,265.935 173.115,266.037 173.93,266.128 174.744,266.209 175.559,266.28 176.374,266.339 177.189,266.388 
 178.004,266.425 178.818,266.451 179.633,266.465 180.448,266.467 181.263,266.457 182.077,266.433 182.892,266.395 183.707,266.344 184.522,266.278 185.336,266.197 
 186.151,266.101 186.966,265.989 187.781,265.861 188.596,265.718 189.41,265.56 190.225,265.386 191.04,265.2 191.855,265.002 192.669,264.795 193.484,264.583 
 194.299,264.369 195.114,264.159 195.928,263.959 196.743,263.776 197.558,263.617 198.373,263.488 199.188,263.394 200.002,263.342 200.817,263.333 201.632,263.369 
 202.447,263.447 203.261,263.565 204.076,263.718 204.891,263.899 205.706,264.103 206.52,264.324 207.335,264.556 208.15,264.795 208.965,265.037 209.78,265.279 
 210.594,265.518 211.409,265.753 212.224,265.983 213.039,266.207 213.853,266.424 214.668,266.634 215.483,266.837 216.298,267.034 217.113,267.224 217.927,267.407 
 218.742,267.584 219.557,267.756 220.372,267.921 221.186,268.081 222.001,268.236 222.816,268.386 223.631,268.531 224.445,268.672 225.26,268.809 226.075,268.941 
 226.

# [MNIST with BP vs DFA](./mnist.ipynb)

In [10]:
@code_native(bp(x,y,n_hidden,max_iter))

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[1]
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r13
	pushq	%r12
	pushq	%rbx
	subq	$808, %rsp              ## imm = 0x328
	movq	%rcx, -808(%rbp)
	movq	%rcx, %r13
	movq	%rdx, -680(%rbp)
	movq	%rsi, %r14
	movq	%r14, -784(%rbp)
	movq	%rdi, %r12
	movq	%r12, -672(%rbp)
	movabsq	$4856052464, %r15       ## imm = 0x121717AF0
	movabsq	$jl_get_ptls_states_fast, %rax
	callq	*%rax
	movq	%rax, %rbx
	movq	%rbx, -664(%rbp)
	leaq	-640(%rbp), %rdi
	movq	$0, -56(%rbp)
	movq	$0, -64(%rbp)
	movq	$0, -72(%rbp)
	movq	$0, -80(%rbp)
	movabsq	$_setcontext, %rax
	movl	$552, %esi              ## imm = 0x228
	callq	*%rax
	movq	$148, -656(%rbp)
	movq	(%rbx), %rax
	movq	%rax, -648(%rbp)
	leaq	-656(%rbp), %rax
	movq	%rax, (%rbx)
	movq	$0, -88(%rbp)
Source line: 35
	movabsq	$initParams, %rax
	movq	%r12, %rdi
	movq	%r14, %rsi
	movq	-680(%rbp), %rdx
	movq	%r13, %rbx
	movq	%rbx, %rcx
	callq	*%rax
	movq	%rax, -640(%rbp)
	movq	(%rax), %rcx
	movq